In [38]:
# A Python connection to the main database
# called "olympus".
import olympusgrpc.pythonclient

ModuleNotFoundError: No module named 'olympusgrpc'

In [42]:
######################################
# TO UPDATE USERNAME AND PASSWORD (SSO)

'''
SSO or Single sign-on enables users to
to log into multiple web applications
with one set of credentials.'''
######################################
# Dictionary data structure because it
# consists of key-value pairs.

# "username" is the key and "" is the value.
properties = {
  "username": "",
  "password": "",
  "ENV": "prod",
  "datasource": "STARBURST",
  "queryTimeout": "3600"
}

'''
I had to downgrade my version of numpy to 1.26.4
because of an incompatibility issue.
pip install numpy==1.26.4
'''
# "np" is an alternative name for the "numpy"
# library. It is an alias.

# "numpy" is a library 
import numpy as np
# import qpython.qconnection
import pandas as pd
import jaydebeapi

'''
Built-in os module with methods for interacting with the
operating system, like creating files and directories,
management of files and directories, input, output,
environment variables, process management, etc.
'''
import os
# import olympusgrpc.pythonclient

'''
The pip install DateTime command installs the DateTime
package, this way the DateTime data type can be made
use of in a Python program.'''
import datetime

'''
Displaying the current working directory shows the
programmer where the program is currently located.

This is optional, but there may be certain instances
where a program needs to know or get the current working
directory. Outputting to the terminal is not always
necessary.
'''

# /home/deniz/Python/King
print(f'os.getcwd(): {os.getcwd()}')

os.getcwd(): /home/deniz/Python/King


In [35]:
# import os
# import pandas as pd

#################################################
#TO UPDATE DATE FOR THE RUN AND DWH BUSINESS DATE
#DHW BUSINESS DATE NEEDS TO BE 1BD BEFORE
#################################################

# year-month-day
date = 20240903

# Previous business date compared to date.
dwh_business_date = "2024-09-02 00:00:00"

# 2024-09-02
# Using string slicing with [:10] to extract the
# the characters located in indices 0 through 9.

# It may be necessary to get the time later on
# somewhere in the program.
dwh_business_date_short = dwh_business_date[:10]

# F-string format:
# print(f'random_text {variable_name}.')

'''
Programs sometimes crash. Knowing the values of variables
can help programmers understand why a program has bugs.

Printing out information about a program and saving it to
a file makes the debugging process easier.

Logging the effect that certain instructions have makes
the program easier to understand.
'''

# E.g.
# print(f'Switzerland GDP per capita: {swiss_gdp_per_capita}')
print(f'\ndwh_business_date_short: {dwh_business_date_short}')


dwh_business_date_short: 2024-09-02


In [36]:
# Calling the function.
def call_olympus_trades_for_month_ptt():
  print(f'os.getcwd(): {os.getcwd()}')

  # Pass the "conn" variable to read_sql() because it
  # must be aware of the connection to the database.
  conn = olympusgrpc.pythonclient.connect(properties)
  
  df = pd.read_sql(
    f"""

    with alert_data as
    (select ph.root_order_id,
        uf.uno_alert_id,
        ph.trade_date,
        ph.pmc_price,
        replace(ph.smcp,'.0',") as smcp,
        ph.buy_sell_indicator,
        ph.trade_unit_price,
        ph.trade_id,
        ph.trade_uit_id,
        ph.segment_description_level_4,
        ph.ppl_product_name,
        ph.trade_currency

    FROM gfolyncr_mis_managed.om_uno_fact uf
    JOIN gfolyrep_work.PanelHistory ph ON ph.alert_id like SUBSTRING(uf.source_id,1,36)
    AND ph.dwh_business_date = cast('{dwh_business_date}' as timestamp)
    where uf.item_definition_id IN ('ID-20230906-135243563','ID-20231013-155037324')
    and uf.uno_alert_status = 'new'
    and ph.excpetion_23b='Y'),

    pmc_prices as
    (select smcp,
    MAX(CASE WHEN derived_method = 'BID' THEN metric_value ELSE NULL END) as price_bid,
    MAX(CASE WHEN derived_method = 'ASK' THEN metric_value ELSE NULL END)as price_ask,
    MAX(CASE WHEN derived_method = 'latest' THEN metric_value ELSE NULL END) as price_last
    from (select smcp, derived_method,metric_value
        from
            (
            select
            smcp,
            derived_method,
            metric_value,
            row_number() OVER(PARTITION by SMCP
            ORDER BY lpad(CASE WHEN price_name LIKE 'PMC-PRICE%%' THEN
    REGEXP_REPLACE(price_name,'PMC-PRICE',"") ELSE NULL END,2,'0')ASC) as RN
            from gfolynref_standardization.OM_PMC_RATES_LKP
            where dwh_business_date = {date}
            and policy_code = 'EOD-NAMR'
            and metric_category='PRICE'
            and policy_category='GoldenMetricDerived'
            and upper(contributor) not like '%%CITI%%'
            and derived_method = 'BID'
            )A
            WHERE RN=1
        UNION ALL
            select smcp,derived_method,metric_value
            from
            (
            select
            smcp,
            derived_method,
            metric_value,
            row_number() OVER(PARTITION BY SCMP)
            ORDER BY lpad(CASE WHEN price_name LIKE 'PMC-PRICE%%' THEN
        REGEXP_REPLACE(price_name,'PMC-PRICE',"") ELSE NULL END,2,'0') ASC) as RN
            from gfolynref_standardization.OM_PMC_RATES_LKP
            where dwh_business_date = {date}
            and policy_code = 'EOD-NAMR'
            and metric_category='PRICE'
            and policy_category='GoldenMetricDerived'
            and upper(contributor) not like '%%CITI%%'
            and derived_method = 'ASK'
            )B
        WHERE RN = 1
        UNION ALL

        --SCMPs where BID and ASK prices are not available

        select smcp, derived_method, metric_value
        from
        (
        select
        smcp,
        'latest' as derived_method,
        metric_value,
        row number() OVER(PARTITION BY SMCP)
        ORDER BY lpad(CASE WHEN price_name LIKE 'PMC-PRICE%%' then)
    REGEXP_REPLACE(price_name,'PMC-PRICE',") ELSE NULL END,2,'0')ASC) AS RN
        from gfolynref_standardization.OM_PM_RATES_LKP
        where dwh_business_date={date}
        and policy_code = 'EOD-NAMR'
        and metric_category='PRICE'
        and policy_category='GoldenMetricDerived'
        and upper(contributor) not like '%%CITI%%'
        )C
        WHERE RN=1
    )D
    where smcp in (select DISTINCT smcp FROM alert_data)
    group by smcp
    )

    select distinct ad.*,pp.price_bid,pp.price_ask,pp.price_last
    from alert_data ad
    join pmc_prices pp on ad.smcp = pp.smcp;

    """,
    conn,
  )
  print(df)

In [37]:
call_olympus_trades_for_month_ptt()

os.getcwd(): /home/deniz/Python/King


NameError: name 'olympusgrpc' is not defined